<a href="https://colab.research.google.com/github/moonlightbotanist/ColoradoView_LST/blob/master/LSTtrends_LandCover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import common python libs**

In [0]:
import os
import sys
import datetime
import math
import csv
import numpy as np                  # to create a sequence for plotting
import pandas as pd                 # for datetime objects and dataframes
import matplotlib.pyplot as plt     # for plotting histograms
from google.colab import drive      # for exporting from distributed machine to GDrive
from google.colab import files

**Install, import, & authenticate earthengine python API**

In [0]:
##@title install earthengine python API

##reference: https:#developers.google.com/earth-engine/python_install_manual
#!pip install google-api-python-client
#!python -c "from oauth2client import crypt"
#!openssl version
!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api

     |████████████████████████████████| 61kB 1.9MB/s 
ERROR: Operation cancelled by user


In [0]:
##@title set up authentication credentials (earthengine)
!earthengine authenticate

# test 1: should not show any error message with the following command
#!python -c "import ee; ee.Initialize()"
# Import the Earth Engine Python Package
import ee
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

# test 2: should print the metadata of the test image
# Print the information for an image asset.
image = ee.Image('srtm90_v4')
print(image.getInfo())

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/vQHnn43br-t64NK45VGbmsNAVLlOpk9bQh9HKjKQH1G

**Beginning of GEE script**
<br>Import assets and data from GEE repository

In [0]:
# Import collections of LST ARD data for each sensor
LS5 = ee.ImageCollection('projects/ColoradoView/Landsat/ARD/5')
LS8 = ee.ImageCollection('projects/ColoradoView/Landsat/ARD/8')

LS7_init = ee.ImageCollection('projects/ColoradoView/Landsat/ARD/7') # Due to issues with 2007 data, filter those data out
LS7_first = LS7_init.filterDate('2000-01-01', '2007-01-01')
LS7_second = LS7_init.filterDate('2008-01-01', '2019-01-01')
LS7 = LS7_first.merge(LS7_second)

# Merge Landsat ARD Collections into the desired aggregates for analysis | LS57, LS78, LS578
LS57 = LS5.merge(LS7)
LS78 = LS7.merge(LS8)
LS578 = LS5.merge(LS7).merge(LS8)

# Colorado Polygon for clipping analysis
CO_Bnds = ee.FeatureCollection('TIGER/2016/States').filter(ee.Filter.eq('GEOID', '08'))
#print(CO_bnds, 'CO_bnds')

# Create geometry of six Colorado counties of interest
US_Counties = ee.FeatureCollection('TIGER/2018/Counties')
countyGEOID_array = ['08031','08111','08047','08081','08039','08095' ];
countyName_array = ['denver','san_juan', 'gilpin','moffat','elbert','phillips'];

denver = countyGEOID_array[0];
sanjuan = countyGEOID_array[1];
gilpin = countyGEOID_array[2]; 
moffat = countyGEOID_array[3]; 
elbert = countyGEOID_array[4]; 
phillips = countyGEOID_array[5]; 

denver_bnds = ee.Feature(US_Counties.filter(ee.Filter.eq('GEOID', denver))); 
sanjuan_bnds = ee.Feature(US_Counties.filter(ee.Filter.eq('GEOID', sanjuan))); 
gilpin_bnds = ee.Feature(US_Counties.filter(ee.Filter.eq('GEOID', gilpin))); 
moffat_bnds = ee.Feature(US_Counties.filter(ee.Filter.eq('GEOID', moffat))); 
elbert_bnds = ee.Feature(US_Counties.filter(ee.Filter.eq('GEOID', elbert))); 
phillips_bnds = ee.Feature(US_Counties.filter(ee.Filter.eq('GEOID', phillips)));

geom_list = ee.List([denver_bnds, sanjuan_bnds, gilpin_bnds, moffat_bnds, elbert_bnds, phillips_bnds])
geom_fc = ee.FeatureCollection(geom_list)

CO_counties = geom_fc.flatten().union().geometry() # produce the union of the combined six counties as a single geometry

# Land cover types from NLCD (b1 = developed, b2 = barren, b3 = forest, b4 = shrubscrub, b5 = pasture, b6 = crops)
NLCD = ee.Image('projects/ColoradoView/Landsat/NLCD_classes')
NLCDlist = NLCD.bandNames()
NLCDlist_cli = NLCD.bandNames().getInfo()
print(NLCDlist_cli)
print(NLCD)

['developed', 'barren', 'forest', 'shrubscrub', 'pasture', 'crops']
ee.Image({
  "type": "Invocation",
  "arguments": {
    "id": "projects/ColoradoView/Landsat/NLCD_classes"
  },
  "functionName": "Image.load"
})


**Run bitwise operators of PixelQA band to mask clouds, shadows, occlusion, etc.**
<BR> This masking matches the masks used when comparing sensors previously.

In [0]:
def maskARDlst(image):
  fillBitMask = 1
  clearBitMask = 1 << 1
  waterBitMask = 1 << 2
  shadowBitMask = 1 << 3
  snowBitMask = 1 << 4
  cloudBitMask = 1 << 5
  occlusionBitMask = 1 << 10
  
  lst_name = 'b1'
  pixelqa_name = 'b2'
  lstqa_name = 'b3'
  lstqa_scale_name = 'STQA_scale_factor'
  
  lstqa_threshold = 5.0;
  
  # Get the pixel QA band
  pixelqa = image.select(pixelqa_name)
  
  # Get the STQA band
  lstqa = image.select(lstqa_name)
  stqa_scale = ee.Number(image.get(lstqa_scale_name))
  
  mask = pixelqa.bitwiseAnd(fillBitMask).eq(0)\
        .And(pixelqa.bitwiseAnd(clearBitMask).neq(0))\
        .And(pixelqa.bitwiseAnd(waterBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(shadowBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(snowBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(cloudBitMask).eq(0))\
        .And(pixelqa.bitwiseAnd(occlusionBitMask).eq(0))\
        .And(lstqa.multiply(stqa_scale).lt(lstqa_threshold))          
        #.and(pixelqa.bitwiseAnd(cloudConfBit7Mask).eq(0))#.and(pixelqa.bitwiseAnd(cirrusConfBit9Mask).eq(0))
              
# Return the masked image (excluding the PIXELQA and STQA layers)
  return image.updateMask(mask).select(lst_name)

# Run the function for each sensor and combination of sensors
L5_masked = LS5.map(maskARDlst)
# print(L5_masked)
L7_masked = LS7.map(maskARDlst)
# print(L7_masked)
L8_masked = LS8.map(maskARDlst)
# print(L8_masked)
L57_masked = LS57.map(maskARDlst)
# print(L57_masked)
L78_masked = LS78.map(maskARDlst)
# print(L78_masked)
L578_masked = LS578.map(maskARDlst)
# print(L578_masked)

**Produce a list of all dates for the period of interest**

In [0]:
# Define the time period of interest
ST_YEAR = 2010
ED_YEAR = 2019
Date_Start = ee.Date(str(ST_YEAR)+'-01-01')
Date_End = ee.Date(str(ED_YEAR)+'-12-31')
Date_window = ee.Number(30)

# Sequence of steps to produce list of dates
n_months = Date_End.difference(Date_Start,'day').round();
dates = ee.List.sequence(0,n_months,1)
def make_datelist(n):
  return Date_Start.advance(n,'day')
dates = dates.map(make_datelist)

# from datetime import datetime
viewDates = dates.getInfo()
print(viewDates)
#print(dates)

[{'type': 'Date', 'value': 1262304000000}, {'type': 'Date', 'value': 1262390400000}, {'type': 'Date', 'value': 1262476800000}, {'type': 'Date', 'value': 1262563200000}, {'type': 'Date', 'value': 1262649600000}, {'type': 'Date', 'value': 1262736000000}, {'type': 'Date', 'value': 1262822400000}, {'type': 'Date', 'value': 1262908800000}, {'type': 'Date', 'value': 1262995200000}, {'type': 'Date', 'value': 1263081600000}, {'type': 'Date', 'value': 1263168000000}, {'type': 'Date', 'value': 1263254400000}, {'type': 'Date', 'value': 1263340800000}, {'type': 'Date', 'value': 1263427200000}, {'type': 'Date', 'value': 1263513600000}, {'type': 'Date', 'value': 1263600000000}, {'type': 'Date', 'value': 1263686400000}, {'type': 'Date', 'value': 1263772800000}, {'type': 'Date', 'value': 1263859200000}, {'type': 'Date', 'value': 1263945600000}, {'type': 'Date', 'value': 1264032000000}, {'type': 'Date', 'value': 1264118400000}, {'type': 'Date', 'value': 1264204800000}, {'type': 'Date', 'value': 1264291

**Produce a list of lists of dates, with the inner list being dates and the outer list being years.**


In [0]:
# Create lists of date sets by year
YEAR_RANGE = ED_YEAR - ST_YEAR
Date_Start = [str(iYr+2010)+'-01-01' for iYr in range(YEAR_RANGE)]
Date_End = [str(iYr+2010)+'-12-31' for iYr in range(YEAR_RANGE)]
Year_Seq = range(ST_YEAR, ED_YEAR)
print(Year_Seq) 

print("startYears", Date_Start)
print("endYears", Date_End)

# Sequence of steps to produce lists of dates for each year
def list_of_dates(start_list, end_list):
  datelist = []
  for i in range(len(start_list)):
    start_date = ee.Date(start_list[i])
    end_date = ee.Date(end_list[i])
    n_months = end_date.difference(start_date,'day').round();
    dates = ee.List.sequence(0,n_months,1)
    def make_datelist(n):
      return start_date.advance(n,'day')
    dates = dates.map(make_datelist)
    dates_cli = dates.getInfo()
    datelist.append(dates_cli)
  return(datelist)

datelist_cli = list_of_dates(Date_Start,Date_End)
datelist_EE = ee.List(datelist_cli)
print(datelist_cli)
print('List of yearly dates length.....', len(datelist_cli))
print('List of 2000 dates length.....', len(datelist_cli[0]))
print('List of 2001 dates length.....',len(datelist_cli[1]))

range(2010, 2019)
startYears ['2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01']
endYears ['2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31']
[[{'type': 'Date', 'value': 1262304000000}, {'type': 'Date', 'value': 1262390400000}, {'type': 'Date', 'value': 1262476800000}, {'type': 'Date', 'value': 1262563200000}, {'type': 'Date', 'value': 1262649600000}, {'type': 'Date', 'value': 1262736000000}, {'type': 'Date', 'value': 1262822400000}, {'type': 'Date', 'value': 1262908800000}, {'type': 'Date', 'value': 1262995200000}, {'type': 'Date', 'value': 1263081600000}, {'type': 'Date', 'value': 1263168000000}, {'type': 'Date', 'value': 1263254400000}, {'type': 'Date', 'value': 1263340800000}, {'type': 'Date', 'value': 1263427200000}, {'type': 'Date', 'value': 1263513600000}, {'type': 'Date', 'value': 1263600000000}, {'type': 'Date', 'value': 12636864

**Prepare images for the NLCD masks.**
<BR>This function takes the masked image collections, filters by date and mosaics the images, converts to images named by date and stores them in a clientside list.

In [0]:
# This function takes the masked image collections, filters by date and mosaics the images, converts to images named by date and stores them in a clientside list
def process_IC_to_daily_I(LS_masked):

  # Create client side list to append annual images with daily bands to
  image_list = []

  # Loop over the items in date range list
  for i in range(len(datelist_cli)):
    datelist_loop = ee.List(datelist_EE.get(i))
    datelist_loop_cli = datelist_loop.getInfo()
    # print('datelist_i.......   ', datelist_loop_cli)

    ############################################################################################
    ## Function to filter by each date and mosaic to create list of daily images for Colorado ##

    def filter_dates_mosaic(date1):
      dateDict = ee.Dictionary(date1)
      start = ee.Date(dateDict.get('value'));
      end = ee.Date(dateDict.get('value')).advance(1,'day')
      date_range = ee.DateRange(start,end)
      images = LS_masked.filterDate(date_range).select('b1').mosaic().set('date', start)
      return(images)

    ############################ Run the function ##################################
    LS_IC = ee.ImageCollection(datelist_loop.map(filter_dates_mosaic))

    ################################################################################
    ################## Function to return dates from IC ############################

    def returnDate(string):
      indexStr = ee.String(string).replace('_b1','')
      indexNum = ee.Number.parse(indexStr)
      datesDicts = ee.Dictionary(datelist_loop.get(indexNum))
      dates = ee.Date(datesDicts.get('value'))
      year = ee.String(dates.get('year'))
      month = ee.String(dates.get('month'))
      day = ee.String(dates.get('day'))
      return(ee.String(year),ee.String(month),ee.String(day))

    # Get bandnames of image
    LS_I = LS_IC.toBands()
    LS_I_bandnames = LS_I.bandNames()

    ############################ Run the function #################################
    LS_dates_toclean = LS_I_bandnames.map(returnDate)
    LS_dates_toclean_cli = LS_dates_toclean.getInfo()
    # print('dates to clean.......   ', LS_dates_toclean_cli)
    # print('length of dates to clean.......   ', len(LS_dates_toclean_cli))

    ################################################################################
    ############# Function to produce datestrings to set as bandnames ##############

    def produce_date_strings(cli_dates):
      # Create flat list of strings for all YMDs
      str_list = []
      for i in cli_dates:
        for j in i:
          str_list.append(str(j))
  
      # Append leading 0s onto single digit months and dates
      lead0_list = []
      for i in str_list:
        if len(i) == 1:
          lead0_list.append('0'+i)
        else:
          lead0_list.append(i)

      # Use list comprehension to group YMD items into list of lists
      str_list_lists = [lead0_list[x:x+3] for x in range(0, len(lead0_list),3)]

      # Produce list of complete date strings
      date_strs= []
      for i in str_list_lists:
        date_strs.append('-'.join(i))
      return(date_strs)

    ############################ Run the function ##################################
    LS_dates_clean_cli = produce_date_strings(LS_dates_toclean_cli)
    # print('datestrings.......   ', LS_dates_clean_cli)
    # print('length of datestrings.......   ', len(LS_dates_clean_cli))

    # Cast dates to ee.List
    LS_dates_EE = ee.List(LS_dates_clean_cli)

    # Rename bands of daily image with date strings
    LS_I_datenames = LS_I.rename(LS_dates_EE)
    LS_I_datenames_bands = LS_I_datenames.bandNames()
    LS_I_datenames_bands_cli = LS_I_datenames_bands.getInfo()
    print('bandnames for image.......   ', LS_I_datenames_bands_cli) 
    print('length of bandnames for image.......   ', len(LS_I_datenames_bands_cli)) 
    print('') 

    # Append the resulting image to the client side list
    image_list.append(LS_I_datenames)  
    
  return(image_list)

In [0]:
# Apply the function to produce annual lists of images with daily bands for landsat 5
L5_daily_I_by_years = process_IC_to_daily_I(L5_masked)

# Check the reulting band names to ensure the proper result
L5_check =  L5_daily_I_by_years[1]
L5_check_bands = L5_check.bandNames()
L5_check_bands_cli = L5_check_bands.getInfo()
print('Resulting band names', L5_check_bands_cli)

In [0]:
# Apply the function to produce annual lists of images with daily bands for landsat 7
L7_daily_I_by_years = process_IC_to_daily_I(L7_masked)

# Check the reulting band names to ensure the proper result
L7_check =  L7_daily_I_by_years[1]
L7_check_bands = L7_check.bandNames()
L7_check_bands_cli = L7_check_bands.getInfo()
print('Resulting band names', L7_check_bands_cli)

In [0]:
# Apply the function to produce annual lists of images with daily bands for landsat 8
L8_daily_I_by_years = process_IC_to_daily_I(L8_masked)

# Check the reulting band names to ensure the proper result
L8_check =  L8_daily_I_by_years[1]
L8_check_bands = L8_check.bandNames()
L8_check_bands_cli = L8_check_bands.getInfo()
print('Resulting band names', L8_check_bands_cli)

In [0]:
# Apply the function to produce annual lists of images with daily bands for landsat 5 and 7
L57_daily_I_by_years = process_IC_to_daily_I(L57_masked)

# Check the reulting band names to ensure the proper result
L57_check =  L57_daily_I_by_years[1]
L57_check_bands = L57_check.bandNames()
L57_check_bands_cli = L57_check_bands.getInfo()
print('Resulting band names', L57_check_bands_cli)

bandnames for image.......    ['2010-01-02', '2010-01-04', '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09', '2010-01-10', '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15', '2010-01-16', '2010-01-17', '2010-01-19', '2010-01-20', '2010-01-21', '2010-01-22', '2010-01-23', '2010-01-24', '2010-01-25', '2010-01-26', '2010-01-27', '2010-01-28', '2010-01-29', '2010-01-30', '2010-01-31', '2010-02-02', '2010-02-03', '2010-02-04', '2010-02-05', '2010-02-06', '2010-02-07', '2010-02-08', '2010-02-09', '2010-02-10', '2010-02-11', '2010-02-12', '2010-02-13', '2010-02-14', '2010-02-15', '2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19', '2010-02-22', '2010-02-24', '2010-02-26', '2010-02-28', '2010-03-01', '2010-03-02', '2010-03-03', '2010-03-04', '2010-03-05', '2010-03-06', '2010-03-07', '2010-03-09', '2010-03-10', '2010-03-11', '2010-03-12', '2010-03-13', '2010-03-15', '2010-03-16', '2010-03-17', '2010-03-18', '2010-03-20', '2010-03-21', '2010-03-22', '2010-03-23', '20

In [0]:
# Apply the function to produce annual lists of images with daily bands for landsat 7 and 8
L78_daily_I_by_years = process_IC_to_daily_I(L78_masked)

# Check the reulting band names to ensure the proper result
L78_check =  L78_daily_I_by_years[1]
L78_check_bands = L78_check.bandNames()
L78_check_bands_cli = L78_check_bands.getInfo()
print('Resulting band names', L78_check_bands_cli)

In [0]:
# Apply the function to produce annual lists of images with daily bands for landsat 5, 7 and 8
L578_daily_I_by_years = process_IC_to_daily_I(L578_masked)

# Check the reulting band names to ensure the proper result
L578_check =  L578_daily_I_by_years[1]
L578_check_bands = L578_check.bandNames()
L578_check_bands_cli = L578_check_bands.getInfo()
print('Resulting band names', L578_check_bands_cli)

bandnames for image.......    ['2010-01-02', '2010-01-04', '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09', '2010-01-10', '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15', '2010-01-16', '2010-01-17', '2010-01-19', '2010-01-20', '2010-01-21', '2010-01-22', '2010-01-23', '2010-01-24', '2010-01-25', '2010-01-26', '2010-01-27', '2010-01-28', '2010-01-29', '2010-01-30', '2010-01-31', '2010-02-02', '2010-02-03', '2010-02-04', '2010-02-05', '2010-02-06', '2010-02-07', '2010-02-08', '2010-02-09', '2010-02-10', '2010-02-11', '2010-02-12', '2010-02-13', '2010-02-14', '2010-02-15', '2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19', '2010-02-22', '2010-02-24', '2010-02-26', '2010-02-28', '2010-03-01', '2010-03-02', '2010-03-03', '2010-03-04', '2010-03-05', '2010-03-06', '2010-03-07', '2010-03-09', '2010-03-10', '2010-03-11', '2010-03-12', '2010-03-13', '2010-03-15', '2010-03-16', '2010-03-17', '2010-03-18', '2010-03-20', '2010-03-21', '2010-03-22', '2010-03-23', '20

**Produce lists of years with inner lists of images based on landcover type.** 
<BR>Each landcover type image has bands corresponding to each date.

In [0]:
# Function to produce list of lists with masked images for each landcover type
def NLCD_filtered_IC(annual_image):
  # Produce masks
  dev_mask = NLCD.select(ee.String(NLCDlist.get(0)))
  bar_mask = NLCD.select(ee.String(NLCDlist.get(1)))
  for_mask = NLCD.select(ee.String(NLCDlist.get(2)))
  shr_mask = NLCD.select(ee.String(NLCDlist.get(3)))
  pas_mask = NLCD.select(ee.String(NLCDlist.get(4)))
  crp_mask = NLCD.select(ee.String(NLCDlist.get(5)))  

  # Produce masked images
  dev_masked = annual_image.updateMask(dev_mask).set('Name', 'Developed')
  bar_masked = annual_image.updateMask(bar_mask).set('Name', 'Barren')
  for_masked = annual_image.updateMask(for_mask).set('Name', 'Forested')
  shr_masked = annual_image.updateMask(shr_mask).set('Name', 'Shrubland and scrubland')
  pas_masked = annual_image.updateMask(pas_mask).set('Name', 'Pasture')
  crp_masked = annual_image.updateMask(crp_mask).set('Name', 'Crops')

  # Create image collection of masked images
  masked_list = [dev_masked, bar_masked, for_masked, shr_masked, pas_masked, crp_masked]
  # masked_IC = ee.ImageCollection(masked_list)
  return(masked_list)

In [0]:
# Apply the function to each annual image in the list with bands corresponding to each day for Landsat 5
L5_I_NLCDmasks = [None] * len(L5_daily_I_by_years)
print(len(L5_I_NLCDmasks))

for i in range(len(L5_daily_I_by_years)):
  L5_I_NLCDmasks[i] = NLCD_filtered_IC(L5_daily_I_by_years[i])

# Investigate function outputs
# Should have a list of yearly image collections with images of landcover with bands for each day—appears to be the case
print('Number of years.... =', len(L5_I_NLCDmasks))
print('Number of NLCD types.... =', len(L5_I_NLCDmasks[0]))
L5_I_check_bands = L5_I_NLCDmasks[0][0]
L5_I_check_name = L5_I_check_bands.get('Name')
L5_I_check_name_cli = L5_I_check_name.getInfo()
L5_I_check_names = L5_I_check_bands.bandNames()
L5_I_check_cli = L5_I_check_names.getInfo()
print('Bandnames of image.... =', L5_I_check_cli)
print('"Name" of image.... =', L5_I_check_name_cli)

In [0]:
# Apply the function to each annual image in the list with bands corresponding to each day for Landsat 7
L7_I_NLCDmasks = [None] * len(L7_daily_I_by_years)
print(len(L7_I_NLCDmasks))

for i in range(len(L7_daily_I_by_years)):
  L7_I_NLCDmasks[i] = NLCD_filtered_IC(L7_daily_I_by_years[i])

# Investigate function outputs
# Should have a list of yearly image collections with images of landcover with bands for each day—appears to be the case
print('Number of years.... =', len(L7_I_NLCDmasks))
print('Number of NLCD types.... =', len(L7_I_NLCDmasks[0]))
L7_I_check_bands = L7_I_NLCDmasks[0][0]
L7_I_check_name = L7_I_check_bands.get('Name')
L7_I_check_name_cli = L7_I_check_name.getInfo()
L7_I_check_names = L7_I_check_bands.bandNames()
L7_I_check_cli = L7_I_check_names.getInfo()
print('Bandnames of image.... =', L7_I_check_cli)
print('"Name" of image.... =', L7_I_check_name_cli)

In [0]:
# Apply the function to each annual image in the list with bands corresponding to each day for Landsat 8
L8_I_NLCDmasks = [None] * len(L8_daily_I_by_years)
print(len(L8_I_NLCDmasks))

for i in range(len(L8_daily_I_by_years)):
  L8_I_NLCDmasks[i] = NLCD_filtered_IC(L8_daily_I_by_years[i])

# Investigate function outputs
# Should have a list of yearly image collections with images of landcover with bands for each day—appears to be the case
print('Number of years.... =', len(L8_I_NLCDmasks))
print('Number of NLCD types.... =', len(L8_I_NLCDmasks[0]))
L8_I_check_bands = L8_I_NLCDmasks[0][0]
L8_I_check_name = L8_I_check_bands.get('Name')
L8_I_check_name_cli = L8_I_check_name.getInfo()
L8_I_check_names = L8_I_check_bands.bandNames()
L8_I_check_cli = L8_I_check_names.getInfo()
print('Bandnames of image.... =', L8_I_check_cli)
print('"Name" of image.... =', L8_I_check_name_cli)

5
Number of years.... = 5
Number of NLCD types.... = 6
Bandnames of image.... = ['2014-01-05', '2014-01-07', '2014-01-09', '2014-01-11', '2014-01-12', '2014-01-14', '2014-01-16', '2014-01-18', '2014-01-21', '2014-01-23', '2014-01-25', '2014-01-28', '2014-02-01', '2014-02-03', '2014-02-06', '2014-02-08', '2014-02-12', '2014-02-13', '2014-02-15', '2014-02-17', '2014-02-19', '2014-02-22', '2014-02-24', '2014-02-26', '2014-02-28', '2014-03-01', '2014-03-03', '2014-03-05', '2014-03-07', '2014-03-08', '2014-03-10', '2014-03-12', '2014-03-14', '2014-03-16', '2014-03-17', '2014-03-19', '2014-03-21', '2014-03-23', '2014-03-26', '2014-03-28', '2014-04-01', '2014-04-02', '2014-04-04', '2014-04-06', '2014-04-08', '2014-04-09', '2014-04-11', '2014-04-13', '2014-04-15', '2014-04-22', '2014-04-24', '2014-04-27', '2014-05-01', '2014-05-03', '2014-05-04', '2014-05-06', '2014-05-08', '2014-05-10', '2014-05-13', '2014-05-15', '2014-05-17', '2014-05-19', '2014-05-20', '2014-05-22', '2014-05-26', '2014-05-

In [0]:
# Apply the function to each annual image in the list with bands corresponding to each day for Landsat 5 and 7
L57_I_NLCDmasks = [None] * len(L57_daily_I_by_years)
print(len(L57_I_NLCDmasks))

for i in range(len(L57_daily_I_by_years)):
  L57_I_NLCDmasks[i] = NLCD_filtered_IC(L57_daily_I_by_years[i])

# Investigate function outputs
# Should have a list of yearly image collections with images of landcover with bands for each day—appears to be the case
print('Number of years.... =', len(L57_I_NLCDmasks))
print('Number of NLCD types.... =', len(L57_I_NLCDmasks[0]))
L57_I_check_bands = L57_I_NLCDmasks[0][0]
L57_I_check_name = L57_I_check_bands.get('Name')
L57_I_check_name_cli = L57_I_check_name.getInfo()
L57_I_check_names = L57_I_check_bands.bandNames()
L57_I_check_cli = L57_I_check_names.getInfo()
print('Bandnames of image.... =', L57_I_check_cli)
print('"Name" of image.... =', L57_I_check_name_cli)

9
Number of years.... = 9
Number of NLCD types.... = 6
Bandnames of image.... = ['2010-01-02', '2010-01-04', '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09', '2010-01-10', '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15', '2010-01-16', '2010-01-17', '2010-01-19', '2010-01-20', '2010-01-21', '2010-01-22', '2010-01-23', '2010-01-24', '2010-01-25', '2010-01-26', '2010-01-27', '2010-01-28', '2010-01-29', '2010-01-30', '2010-01-31', '2010-02-02', '2010-02-03', '2010-02-04', '2010-02-05', '2010-02-06', '2010-02-07', '2010-02-08', '2010-02-09', '2010-02-10', '2010-02-11', '2010-02-12', '2010-02-13', '2010-02-14', '2010-02-15', '2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19', '2010-02-22', '2010-02-24', '2010-02-26', '2010-02-28', '2010-03-01', '2010-03-02', '2010-03-03', '2010-03-04', '2010-03-05', '2010-03-06', '2010-03-07', '2010-03-09', '2010-03-10', '2010-03-11', '2010-03-12', '2010-03-13', '2010-03-15', '2010-03-16', '2010-03-17', '2010-03-18', '2010-03-

In [0]:
# Apply the function to each annual image in the list with bands corresponding to each day for Landsat 7 and 8
L78_I_NLCDmasks = [None] * len(L78_daily_I_by_years)
print(len(L78_I_NLCDmasks))

for i in range(len(L78_daily_I_by_years)):
  L78_I_NLCDmasks[i] = NLCD_filtered_IC(L78_daily_I_by_years[i])

# Investigate function outputs
# Should have a list of yearly image collections with images of landcover with bands for each day—appears to be the case
print('Number of years.... =', len(L78_I_NLCDmasks))
print('Number of NLCD types.... =', len(L78_I_NLCDmasks[0]))
L78_I_check_bands = L78_I_NLCDmasks[0][0]
L78_I_check_name = L78_I_check_bands.get('Name')
L78_I_check_name_cli = L78_I_check_name.getInfo()
L78_I_check_names = L78_I_check_bands.bandNames()
L78_I_check_cli = L78_I_check_names.getInfo()
print('Bandnames of image.... =', L78_I_check_cli)
print('"Name" of image.... =', L78_I_check_name_cli)

In [0]:
# Apply the function to each annual image in the list with bands corresponding to each day for Landsat 5, 7, and 8
L578_I_NLCDmasks = [None] * len(L578_daily_I_by_years)
print(len(L578_I_NLCDmasks))

for i in range(len(L578_daily_I_by_years)):
  L578_I_NLCDmasks[i] = NLCD_filtered_IC(L578_daily_I_by_years[i])

# Investigate function outputs
# Should have a list of yearly image collections with images of landcover with bands for each day—appears to be the case
print('Number of years.... =', len(L578_I_NLCDmasks))
print('Number of NLCD types.... =', len(L578_I_NLCDmasks[0]))
L578_I_check_bands = L578_I_NLCDmasks[0][0]
L578_I_check_name = L578_I_check_bands.get('Name')
L578_I_check_name_cli = L578_I_check_name.getInfo()
L578_I_check_names = L578_I_check_bands.bandNames()
L578_I_check_cli = L578_I_check_names.getInfo()
print('Bandnames of image.... =', L578_I_check_cli)
print('"Name" of image.... =', L578_I_check_name_cli)

9
Number of years.... = 9
Number of NLCD types.... = 6
Bandnames of image.... = ['2010-01-02', '2010-01-04', '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09', '2010-01-10', '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15', '2010-01-16', '2010-01-17', '2010-01-19', '2010-01-20', '2010-01-21', '2010-01-22', '2010-01-23', '2010-01-24', '2010-01-25', '2010-01-26', '2010-01-27', '2010-01-28', '2010-01-29', '2010-01-30', '2010-01-31', '2010-02-02', '2010-02-03', '2010-02-04', '2010-02-05', '2010-02-06', '2010-02-07', '2010-02-08', '2010-02-09', '2010-02-10', '2010-02-11', '2010-02-12', '2010-02-13', '2010-02-14', '2010-02-15', '2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19', '2010-02-22', '2010-02-24', '2010-02-26', '2010-02-28', '2010-03-01', '2010-03-02', '2010-03-03', '2010-03-04', '2010-03-05', '2010-03-06', '2010-03-07', '2010-03-09', '2010-03-10', '2010-03-11', '2010-03-12', '2010-03-13', '2010-03-15', '2010-03-16', '2010-03-17', '2010-03-18', '2010-03-

_______________________________________________________________________________
**Exporting statistics for landcover types**

***Calculating and exporting CSVs of means***

In [0]:
def calc_mean_exp(list_of_lists_of_images, sensor_str): 
  for i in range(len(list_of_lists_of_images)):
    for j in range(len(list_of_lists_of_images[i])):
      mean_dict = list_of_lists_of_images[i][j].reduceRegion(reducer = ee.Reducer.mean(),geometry = CO_counties,scale = 30,maxPixels = 1e13)
      mean_feature = ee.Feature(None,mean_dict)
      mean_fc = ee.FeatureCollection(mean_feature)
      landcover_type = ee.String(list_of_lists_of_images[i][j].get('Name'))
      landcover_cli = landcover_type.getInfo()
      landcover_str = str(landcover_cli)
      year = str(Year_Seq[i])
      mean_exp_task = ee.batch.Export.table.toDrive(collection = mean_fc,
                                                  description = sensor_str + '_' + 'mean' + '_' + landcover_str + '_' + year, 
                                                  fileNamePrefix = sensor_str + '_' + 'mean' + '_' + landcover_str + '_' + year,
                                                  fileFormat = 'CSV')
      mean_exp_task.start()

In [0]:
# Calculate CSVs of means for Landsat 5
calc_mean_exp(L5_I_NLCDmasks, 'L5')

In [0]:
# Calculate CSVs of means for Landsat 7
calc_mean_exp(L7_I_NLCDmasks, 'L7')

In [0]:
# Calculate CSVs of means for Landsat 8
calc_mean_exp(L8_I_NLCDmasks, 'L8')

In [0]:
# Calculate CSVs of means for Landsat 5 and 7
calc_mean_exp(L57_I_NLCDmasks, 'L57')

In [0]:
# Calculate CSVs of means for Landsat 7 and 8
calc_mean_exp(L78_I_NLCDmasks, 'L78')

In [0]:
# Calculate CSVs of means for Landsat 5
calc_mean_exp(L578_I_NLCDmasks, 'L578')

***Calculating and exporting CSVs of medians***

In [0]:
def calc_median_exp(list_of_lists_of_images, sensor_str): 
  for i in range(len(list_of_lists_of_images)):
    for j in range(len(list_of_lists_of_images[i])):
      median_dict = list_of_lists_of_images[i][j].reduceRegion(reducer = ee.Reducer.median(),geometry = CO_counties,scale = 30,maxPixels = 1e13)
      median_feature = ee.Feature(None,median_dict)
      median_fc = ee.FeatureCollection(median_feature)
      landcover_type = ee.String(list_of_lists_of_images[i][j].get('Name'))
      landcover_cli = landcover_type.getInfo()
      landcover_str = str(landcover_cli)
      year = str(Year_Seq[i])
      median_exp_task = ee.batch.Export.table.toDrive(collection = median_fc,
                                                  description = sensor_str + '_' + 'median' + '_' + landcover_str + '_' + year, 
                                                  fileNamePrefix = sensor_str + '_' + 'median' + '_' + landcover_str + '_' + year,
                                                  fileFormat = 'CSV')
      median_exp_task.start()

In [0]:
# Calculate CSVs of median for Landsat 5
calc_median_exp(L5_I_NLCDmasks, 'L5')

In [0]:
# Calculate CSVs of median for Landsat 7
calc_median_exp(L7_I_NLCDmasks, 'L7')

In [0]:
# Calculate CSVs of median for Landsat 8
calc_median_exp(L8_I_NLCDmasks, 'L8')

In [0]:
# Calculate CSVs of median for Landsat 5 and 7
calc_median_exp(L57_I_NLCDmasks, 'L57')

In [0]:
# Calculate CSVs of median for Landsat 7 and 8
calc_median_exp(L78_I_NLCDmasks, 'L78')

In [0]:
# Calculate CSVs of median for Landsat 5, 7, and 8
calc_median_exp(L578_I_NLCDmasks, 'L578')

***Calculating and exporting CSVs of modes***

In [0]:
def calc_mode_exp(list_of_lists_of_images, sensor_str): 
  for i in range(len(list_of_lists_of_images)):
    for j in range(len(list_of_lists_of_images[i])):
      mode_dict = list_of_lists_of_images[i][j].reduceRegion(reducer = ee.Reducer.mode(),geometry = CO_counties,scale = 30,maxPixels = 1e13)
      mode_feature = ee.Feature(None,mode_dict)
      mode_fc = ee.FeatureCollection(mode_feature)
      landcover_type = ee.String(list_of_lists_of_images[i][j].get('Name'))
      landcover_cli = landcover_type.getInfo()
      landcover_str = str(landcover_cli)
      year = str(Year_Seq[i])
      mode_exp_task = ee.batch.Export.table.toDrive(collection = mode_fc,
                                                  description = sensor_str + '_' + 'mode' + '_' + landcover_str + '_' + year, 
                                                  fileNamePrefix = sensor_str + '_' + 'mode' + '_' + landcover_str + '_' + year,
                                                  fileFormat = 'CSV')
      mode_exp_task.start()

In [0]:
# Calculate CSVs of mode for Landsat 5
calc_mode_exp(L5_I_NLCDmasks, 'L5')

In [0]:
# Calculate CSVs of mode for Landsat 7
calc_mode_exp(L7_I_NLCDmasks, 'L7')

In [0]:
# Calculate CSVs of mode for Landsat 8
calc_mode_exp(L8_I_NLCDmasks, 'L8')

In [0]:
# Calculate CSVs of mode for Landsat 5 and 7
calc_mode_exp(L57_I_NLCDmasks, 'L57')

In [0]:
# Calculate CSVs of mode for Landsat 7 and 8
calc_mode_exp(L78_I_NLCDmasks, 'L78')

In [0]:
# Calculate CSVs of mode for Landsat 5, 7 and 8
calc_mode_exp(L578_I_NLCDmasks, 'L578')

***Calculating and exporting CSVs of mins***

In [0]:
def calc_min_exp(list_of_lists_of_images, sensor_str): 
  for i in range(len(list_of_lists_of_images)):
    for j in range(len(list_of_lists_of_images[i])):
      min_dict = list_of_lists_of_images[i][j].reduceRegion(reducer = ee.Reducer.min(),geometry = CO_counties,scale = 30,maxPixels = 1e13)
      min_feature = ee.Feature(None,min_dict)
      min_fc = ee.FeatureCollection(min_feature)
      landcover_type = ee.String(list_of_lists_of_images[i][j].get('Name'))
      landcover_cli = landcover_type.getInfo()
      landcover_str = str(landcover_cli)
      year = str(Year_Seq[i])
      min_exp_task = ee.batch.Export.table.toDrive(collection = min_fc,
                                                  description = sensor_str + '_' + 'min' + '_' + landcover_str + '_' + year, 
                                                  fileNamePrefix = sensor_str + '_' + 'min' + '_' + landcover_str + '_' + year,
                                                  fileFormat = 'CSV')
      min_exp_task.start()

In [0]:
# Calculate CSVs of min for Landsat 5
calc_min_exp(L5_I_NLCDmasks, 'L5')

In [0]:
# Calculate CSVs of min for Landsat 7
calc_min_exp(L7_I_NLCDmasks, 'L7')

In [0]:
# Calculate CSVs of min for Landsat 8
calc_min_exp(L8_I_NLCDmasks, 'L8')

In [0]:
# Calculate CSVs of min for Landsat 5 and 7
calc_min_exp(L57_I_NLCDmasks, 'L57')

In [0]:
# Calculate CSVs of min for Landsat 7 and 8
calc_min_exp(L78_I_NLCDmasks, 'L78')

In [0]:
# Calculate CSVs of min for Landsat 5, 7, and 8
calc_min_exp(L578_I_NLCDmasks, 'L578')

***Calculating and exporting CSVs of maxs***

In [0]:
def calc_max_exp(list_of_lists_of_images, sensor_str): 
  for i in range(len(list_of_lists_of_images)):
    for j in range(len(list_of_lists_of_images[i])):
      max_dict = list_of_lists_of_images[i][j].reduceRegion(reducer = ee.Reducer.max(),geometry = CO_counties,scale = 30,maxPixels = 1e13)
      max_feature = ee.Feature(None,max_dict)
      max_fc = ee.FeatureCollection(max_feature)
      landcover_type = ee.String(list_of_lists_of_images[i][j].get('Name'))
      landcover_cli = landcover_type.getInfo()
      landcover_str = str(landcover_cli)
      year = str(Year_Seq[i])
      max_exp_task = ee.batch.Export.table.toDrive(collection = max_fc,
                                                  description = sensor_str + '_' + 'max' + '_' + landcover_str + '_' + year, 
                                                  fileNamePrefix = sensor_str + '_' + 'max' + '_' + landcover_str + '_' + year,
                                                  fileFormat = 'CSV')
      max_exp_task.start()

In [0]:
# Calculate CSVs of max for Landsat 5
calc_max_exp(L5_I_NLCDmasks, 'L5')

In [0]:
# Calculate CSVs of max for Landsat 7
calc_max_exp(L7_I_NLCDmasks, 'L7')

In [0]:
# Calculate CSVs of max for Landsat 8
calc_max_exp(L8_I_NLCDmasks, 'L8')

In [0]:
# Calculate CSVs of max for Landsat 5 and 7
calc_max_exp(L57_I_NLCDmasks, 'L57')

In [0]:
# Calculate CSVs of max for Landsat 7 and 8
calc_max_exp(L78_I_NLCDmasks, 'L78')

In [0]:
# Calculate CSVs of max for Landsat 5, 7, and 8
calc_max_exp(L578_I_NLCDmasks, 'L578')

***Calculating and exporting CSVs of stdDevs***

In [0]:
def calc_stdDev_exp(list_of_lists_of_images, sensor_str): 
  for i in range(len(list_of_lists_of_images)):
    for j in range(len(list_of_lists_of_images[i])):
      stdDev_dict = list_of_lists_of_images[i][j].reduceRegion(reducer = ee.Reducer.stdDev(),geometry = CO_counties,scale = 30,maxPixels = 1e13)
      stdDev_feature = ee.Feature(None,stdDev_dict)
      stdDev_fc = ee.FeatureCollection(stdDev_feature)
      landcover_type = ee.String(list_of_lists_of_images[i][j].get('Name'))
      landcover_cli = landcover_type.getInfo()
      landcover_str = str(landcover_cli)
      year = str(Year_Seq[i])
      stdDev_exp_task = ee.batch.Export.table.toDrive(collection = stdDev_fc,
                                                  description = sensor_str + '_' + 'stdDev' + '_' + landcover_str + '_' + year, 
                                                  fileNamePrefix = sensor_str + '_' + 'stdDev' + '_' + landcover_str + '_' + year,
                                                  fileFormat = 'CSV')
      stdDev_exp_task.start()

In [0]:
# Calculate CSVs of stdDev for Landsat 5
calc_stdDev_exp(L5_I_NLCDmasks, 'L5')

In [0]:
# Calculate CSVs of stdDev for Landsat 7
calc_stdDev_exp(L7_I_NLCDmasks, 'L7')

In [0]:
# Calculate CSVs of stdDev for Landsat 8
calc_stdDev_exp(L8_I_NLCDmasks, 'L8')

In [0]:
# Calculate CSVs of stdDev for Landsat 5 and 7
calc_stdDev_exp(L57_I_NLCDmasks, 'L57')

In [0]:
# Calculate CSVs of stdDev for Landsat 7 and 8
calc_stdDev_exp(L78_I_NLCDmasks, 'L78')

In [0]:
# Calculate CSVs of stdDev for Landsat 5, 7, and 8
calc_stdDev_exp(L578_I_NLCDmasks, 'L578')

________________________________________________________________________________
**Importing and processing CSVs of landcover statistics**
<BR> Mount Google Drive for importing CSVs

In [0]:
drive.mount('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Load lists of CSVs as Pandas dataframes for each sensor from Google Drive and list by landcover type**


In [0]:
# Get lists of full paths of files in directories
# Produce lists of Pandas dataframes for each of the statistics in a dictionary of with keys of landcover to store all of the imports
# For example: For Landsat 8, this function produces dictionary with keys for each land use type (6) and then a single dataframe with columns of each of the statistics
# So, each timeseries of CSVs for each land use type and each statistic is nested two layers deep.
# Thus, there are be 3 parent dictionaries (L5_dict, L7_dict, L8_dict) which will be combined later to produce L57, L78, and L578

import pathlib

def dicts_of_paths(sensor_str): # sensor_str is just number combo. i.e. '5' or '57'

  # Lists to loop over in function
  nlcd_folders = ['Bar', 'Cro', 'Dev', 'For', 'Pas', 'Shr']
  stat_folders = ['Max', 'Mean', 'Med', 'Min', 'Mode', 'StD']

  # Function to get list of posixpath objects for each folder
  def get_file_list(in_dir):
    out_list = []
    for filepath in pathlib.Path(in_dir).glob('*.csv'):
      out_list.append(filepath.absolute())
    return out_list

  sensor_dict = {} # outer dicitonary
  for i_LC in nlcd_folders:
    sensor_dict[i_LC] = {} # inner list to be populated with nlcd_dict
    nlcd_list = []  # inner list
    for i_ST in range(len(stat_folders)):
      path = '/content/gdrive/My Drive/ARD_LST_CO/NLCD_ARD_Stats/' + str(sensor_str) + '/' + str(i_LC) + '/' + str(stat_folders[i_ST])
      fileList = get_file_list(path)

      pdList = [] # create list to accumulated Pandas dataframes as CSVs are being ingested
      for i_File in range(len(fileList)):
        pdList.append(pd.read_csv(fileList[i_File]).iloc[:,1:-1].T.rename(columns = {0: stat_folders[i_ST]})) # Read csvs as pds dfs, remove 'system:index' columns, and transpose the dataframes writing columns as rows

      pdConcat = pd.concat(pdList) # Concatenate the list of dataframes into single timeseries dataframe, reset the index, and drop the previous index
      nlcd_list.append(pdConcat) # append each dataframe to a list to be column binded together with each other.

      pdStats = pd.concat(nlcd_list, axis=1).div(10) # Concatenate across all of the columns to create dataframe with timeseries for each statistic and divide by 10 to normalize the kelvin values

      sensor_dict[i_LC] = pdStats
  return(sensor_dict)

In [0]:
# Run the dicts_of_paths functions for each sensor's CSVs
# L5_dict_pds = dicts_of_paths(5)
# L7_dict_pds = dicts_of_paths(7)
# L8_dict_pds = dicts_of_paths(8)
L578_dict_pds = dicts_of_paths(578)

print('L578_keys', L578_dict_pds.keys())
print('L578_for_keys', type(L578_dict_pds['For']))
print(L578_dict_pds['For'])

L578_keys dict_keys(['Bar', 'Cro', 'Dev', 'For', 'Pas', 'Shr'])
L578_for_keys <class 'pandas.core.frame.DataFrame'>
              Max        Mean    Med    Min   Mode       StD
2000-01-02  274.7  268.690698  268.0  265.2  268.0  2.574726
2000-01-04    NaN         NaN    NaN    NaN    NaN       NaN
2000-01-06    NaN         NaN    NaN    NaN    NaN       NaN
2000-01-07  267.4  264.118125  263.8  262.3  263.8  0.873573
2000-01-08  271.9  263.164033  263.3  259.5  262.7  1.420506
...           ...         ...    ...    ...    ...       ...
2018-12-24    NaN         NaN    NaN    NaN    NaN       NaN
2018-12-25    NaN         NaN    NaN    NaN    NaN       NaN
2018-12-27  272.5  260.445094  260.5  252.8  260.7  1.697619
2018-12-29  277.0  271.242353  271.5  264.4  271.9  1.720388
2018-12-31    NaN         NaN    NaN    NaN    NaN       NaN

[5822 rows x 6 columns]


Export each the statistics dataframes to CSVs

In [0]:
def export_csvs(sensor_dict, sensor_str):
  for iKey in sensor_dict.keys():
    file_exp = sensor_dict[iKey]
    file_exp.to_csv(sensor_str + '_' + iKey + '.csv')
    files.download(sensor_str + '_' + iKey + '.csv')

# export_csvs(L5_dict_pds, 'L5_stats')
# export_csvs(L7_dict_pds, 'L7_stats')
# export_csvs(L8_dict_pds, 'L8_stats')
export_csvs(L578_dict_pds, 'L578_stats')